In [20]:
place = 'los angeles'
keyword = 'android developer'

In [52]:
def get_job_list(place, keyword):
    import requests
    from bs4 import BeautifulSoup
    import time
    
    headers = {
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
        'accept-encoding': 'gzip, deflate, sdch, br',
        'accept-language': 'en-GB,en-US;q=0.8,en;q=0.6',
        'referer': 'https://www.glassdoor.com/',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/51.0.2704.79 Chrome/51.0.2704.79 Safari/537.36',
        'Cache-Control': 'no-cache',
        'Connection': 'keep-alive'
    }
    
    location_headers = {
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.01',
        'accept-encoding': 'gzip, deflate, sdch, br',
        'accept-language': 'en-GB,en-US;q=0.8,en;q=0.6',
        'referer': 'https://www.glassdoor.com/',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/51.0.2704.79 Chrome/51.0.2704.79 Safari/537.36',
        'Cache-Control': 'no-cache',
        'Connection': 'keep-alive'
    }

    data = {"term": place,
            "maxLocationsToReturn": 10}
    
    location_url = "https://www.glassdoor.co.in/findPopularLocationAjax.htm?"
    
    try:
        # get location id for search location
        location_response = requests.post(location_url, headers=location_headers, data=data).json()
        place_id = location_response[0]['locationId']
        job_list_url = 'https://www.glassdoor.com/Job/jobs.htm'
        absolute_url = 'https://www.glassdoor.com'
        data = {
            'clickSource': 'searchBtn',
            'sc.keyword': keyword,
            'locT': 'C',
            'locId': place_id,
            'jobType': ''
        }
        
        result_list = list()
        
        if place_id:
            for i in range(30):     # don't know why, the website can only show the first 30 pages
                response = requests.post(job_list_url, headers=headers, data=data)
                results_page = BeautifulSoup(response.content,'lxml')
                jobs = results_page.find('', class_='jlGrid hover').find_all('li')
                
                for job in jobs:
                    # get the attributes of the job
                    job_name = job.find('div', class_='flexbox jobTitle').find('a').get_text()
                    job_url = absolute_url+job.find('div', class_='flexbox jobTitle').find('a').get('href')
                    company_location = job.find('div', class_='flexbox empLoc').find('div').get_text().split(' – ')
                    job_company = company_location[0][1:]
                    job_city = company_location[1].split(', ')[0]
                    job_state = company_location[1].split(', ')[1]
                    if job.find('span', class_='green small'):
                        job_salary = job.find('span', class_='green small').get_text()[1:]
                    else:
                        job_salary = ''
                    result_list.append((job_name, job_company, job_city, job_state, job_salary, job_url))
                    
                if not results_page.find('li', class_='next'):
                    break   # only one page
                if not results_page.find('li', class_='next').find('a'):
                    break   # reach the end page
                # next page
                job_list_url = absolute_url + results_page.find('li', class_='next').find('a').get('href')
                time.sleep(2)
                    
            return result_list         
        else:
            print('Location not available.')
              
    except:
        print('Fail to find jobs.')

In [53]:
result = get_job_list('Santa Barbara', 'analyst')

In [54]:
result

[('Sr Business System Analyst (MM/PP)',
  'Arthrex',
  'Santa Barbara',
  'CA',
  '$87k-$139k(Glassdoor Est.)',
  'https://www.glassdoor.com/partner/jobListing.htm?pos=101&ao=393346&s=149&guid=000001675cfab2989502318c61d7502d&src=GD_JOB_AD&t=SRFJ&extid=4&exst=OL&ist=&ast=OL&vt=w&slr=true&rtp=0&cs=1_20caeec7&cb=1543453193369&jobListingId=2988013904'),
 ('Data Analyst',
  'AppFolio',
  'Santa Barbara',
  'CA',
  '$42k-$58k(Glassdoor Est.)',
  'https://www.glassdoor.com/partner/jobListing.htm?pos=102&ao=304937&s=149&guid=000001675cfab2989502318c61d7502d&src=GD_JOB_AD&t=SRFJ&extid=4&exst=OL&ist=&ast=OL&vt=w&slr=true&rtp=0&cs=1_c0b0becd&cb=1543453193371&jobListingId=2988134048'),
 ('Data Analytics Analyst',
  'Toyon Research Corporation',
  'Goleta',
  'CA',
  '$80k-$127k(Glassdoor Est.)',
  'https://www.glassdoor.com/partner/jobListing.htm?pos=101&ao=85058&s=58&guid=000001675cfab298a83c894957dc825f&src=GD_JOB_AD&t=SR&extid=1&exst=OL&ist=&ast=OL&vt=w&slr=true&rtp=0&cs=1_634f0b74&cb=15434531